# Task Status Checker Test Notebook

This notebook tests the TaskStatusChecker, which is now a simple deterministic function that doesn't use LLM calls.

**Important**: Run all cells in order from the beginning. The cells depend on variables defined in previous cells.

In [1]:
import os
import sys
import asyncio
from pathlib import Path
from typing import Dict, Any, List

sys.path.append('../src')

# Import required components
from keyvalue_memory import KeyValueMemory
from task_status_checker import TaskStatusChecker, TaskStatusCheckerArgs
from query_tree_manager import QueryTreeManager
from node_history_manager import NodeHistoryManager
from memory_content_types import QueryNode, NodeStatus, QueryMapping, TableMapping, ColumnMapping

## 1. Initialize Memory and Managers

In [2]:
# Initialize shared memory
memory = KeyValueMemory()

# Initialize managers
tree_manager = QueryTreeManager(memory)
task_checker = TaskStatusChecker(memory)

# Clear and create a simple tree
await memory.clear()
root_id = await tree_manager.initialize("Find all students")

# Check status using TaskStatusCheckerArgs
args = TaskStatusCheckerArgs(task="Check single unprocessed node")
result = await task_checker.run(args)
print("Single unprocessed node:")
print(result)
print("\nExpected: ACTION: PROCESS NODE for the root node")

Single unprocessed node:
STATUS REPORT:
- Total nodes: 1
- Processed with good results: 0
- Needs processing: 1
- Needs improvement: 0
CURRENT NODE CONTENT:
- Node ID: node_1748279401.265549_root
- Intent: Find all students
- Status: created
- Parent ID: None
- Children: []
- Mapping:

ACTION: PROCESS NODE: node_1748279401.265549_root needs SQL generation for 'Find all students...'

Expected: ACTION: PROCESS NODE for the root node


## 2. Test Scenario 1: Empty Tree

In [3]:
# Create child nodes properly using QueryNode objects
import time

# Clear and create a complex tree
await memory.clear()

# Create root node
root_id = await tree_manager.initialize("Complex query")

# Create first child node
child1_id = f"node_{time.time()}_1"
node1 = QueryNode(
    nodeId=child1_id,
    intent="Filter students",
    parentId=root_id,
    mapping=QueryMapping(
        tables=[TableMapping(name="students", alias="s")],
        columns=[
            ColumnMapping(table="students", column="id", usedFor="select"),
            ColumnMapping(table="students", column="name", usedFor="select"),
            ColumnMapping(table="students", column="grade", usedFor="filter")
        ]
    )
)
await tree_manager.add_node(node1, root_id)

# Create second child node
child2_id = f"node_{time.time()}_2"
node2 = QueryNode(
    nodeId=child2_id,
    intent="Calculate average",
    parentId=root_id,
    mapping=QueryMapping(
        tables=[TableMapping(name="students", alias="s")],
        columns=[ColumnMapping(table="students", column="grade", usedFor="aggregate")]
    )
)
await tree_manager.add_node(node2, root_id)

# Update node1 with SQL and good result
node1.sql = "SELECT * FROM students WHERE grade > 90"
node1.executionResult = {
    "status": "success", 
    "rowCount": 5, 
    "data": [
        {"id": 1, "name": "John", "grade": 95}, 
        {"id": 2, "name": "Jane", "grade": 92}
    ]
}
node1.status = NodeStatus.EXECUTED_SUCCESS
await tree_manager.update_node(node1.nodeId, node1)

# Store analysis for node1
await memory.set(f"node_{child1_id}_analysis", {
    "result_quality": "excellent",
    "answers_intent": "yes"
})

# Update node2 with SQL but poor result
node2.sql = "SELECT AVG(grade) FROM students"
node2.executionResult = {
    "status": "success", 
    "rowCount": 1, 
    "data": [{"avg_grade": None}]
}
node2.status = NodeStatus.EXECUTED_SUCCESS
await tree_manager.update_node(node2.nodeId, node2)

# Store analysis for node2
await memory.set(f"node_{child2_id}_analysis", {
    "result_quality": "poor",
    "answers_intent": "no"
})

# Set current node to node2
await tree_manager.set_current_node_id(child2_id)

# Check status with TaskStatusCheckerArgs
args = TaskStatusCheckerArgs(task="Check multiple nodes with different states")
result = await task_checker.run(args)
print("Multiple nodes with different states:")
print(result)
print("\nExpected: Shows detailed current node content for node2 with poor quality")

Multiple nodes with different states:
STATUS REPORT:
- Total nodes: 3
- Processed with good results: 1
- Needs processing: 1
- Needs improvement: 1
CURRENT NODE CONTENT:
- Node ID: node_1748279401.2734153_2
- Intent: Calculate average
- Status: executed_success
- Parent ID: node_1748279401.27327_root
- Children: []
- Mapping:
  - Tables: students
  - Columns: students.grade
- SQL: SELECT AVG(grade) FROM students
- Execution Result:
  - Status: success
  - Row count: 1
  - First row: {'avg_grade': None}...
- Evaluation Quality: poor

ACTION: PROCESS NODE: node_1748279401.27327_root needs SQL generation for 'Complex query...'

Expected: Shows detailed current node content for node2 with poor quality


## 6. Test Current Node Update Feature

In [4]:
# Clear and create tree with unprocessed nodes
await memory.clear()
root_id = await tree_manager.initialize("Main query")

# Add multiple unprocessed children - must use QueryNode objects
child1_id = f"node_{time.time()}_child1"
child1 = QueryNode(
    nodeId=child1_id,
    intent="First subquery",
    parentId=root_id,
    mapping=QueryMapping()
)
await tree_manager.add_node(child1, root_id)

child2_id = f"node_{time.time()}_child2"
child2 = QueryNode(
    nodeId=child2_id,
    intent="Second subquery",
    parentId=root_id,
    mapping=QueryMapping()
)
await tree_manager.add_node(child2, root_id)

child3_id = f"node_{time.time()}_child3"
child3 = QueryNode(
    nodeId=child3_id,
    intent="Third subquery",
    parentId=root_id,
    mapping=QueryMapping()
)
await tree_manager.add_node(child3, root_id)

# Check current node before
current_before = await tree_manager.get_current_node_id()
print(f"Current node before: {current_before}")

# Run task checker with TaskStatusCheckerArgs
args = TaskStatusCheckerArgs(task="Check node update feature")
result = await task_checker.run(args)
print("\nTask checker result:")
print(result)

# Check current node after
current_after = await tree_manager.get_current_node_id()
print(f"\nCurrent node after: {current_after}")
print("\nThe current node should have been updated to the recommended node")

Current node before: node_1748279401.279802_root

Task checker result:
STATUS REPORT:
- Total nodes: 4
- Processed with good results: 0
- Needs processing: 4
- Needs improvement: 0
CURRENT NODE CONTENT:
- Node ID: node_1748279401.279802_root
- Intent: Main query
- Status: created
- Parent ID: None
- Children: ['node_1748279401.2798421_child1', 'node_1748279401.2799098_child2', 'node_1748279401.2799687_child3']
- Mapping:

ACTION: PROCESS NODE: node_1748279401.2798421_child1 needs SQL generation for 'First subquery...'

Current node after: node_1748279401.2798421_child1

The current node should have been updated to the recommended node


## 7. Test Priority: Leaf Nodes First

In [5]:
# Clear and create tree with parent-child hierarchy
await memory.clear()
root_id = await tree_manager.initialize("Main query")

# Add parent node
parent_id = f"node_{time.time()}_parent"
parent_node = QueryNode(
    nodeId=parent_id,
    intent="Parent query",
    parentId=root_id,
    mapping=QueryMapping()
)
await tree_manager.add_node(parent_node, root_id)

# Add children to parent
grandchild1_id = f"node_{time.time()}_gc1"
grandchild1 = QueryNode(
    nodeId=grandchild1_id,
    intent="Grandchild 1",
    parentId=parent_id,
    mapping=QueryMapping()
)
await tree_manager.add_node(grandchild1, parent_id)

grandchild2_id = f"node_{time.time()}_gc2"
grandchild2 = QueryNode(
    nodeId=grandchild2_id,
    intent="Grandchild 2",
    parentId=parent_id,
    mapping=QueryMapping()
)
await tree_manager.add_node(grandchild2, parent_id)

# Run task checker with TaskStatusCheckerArgs
args = TaskStatusCheckerArgs(task="Check leaf node priority")
result = await task_checker.run(args)
print("Tree with hierarchy:")
print(result)

# Check which node was selected
current = await tree_manager.get_current_node_id()
print(f"\nSelected node: {current}")
print("\nExpected: Should select a leaf node (grandchild) first, not the parent")

Tree with hierarchy:
STATUS REPORT:
- Total nodes: 4
- Processed with good results: 0
- Needs processing: 4
- Needs improvement: 0
CURRENT NODE CONTENT:
- Node ID: node_1748279401.28578_root
- Intent: Main query
- Status: created
- Parent ID: None
- Children: ['node_1748279401.285818_parent']
- Mapping:

ACTION: PROCESS NODE: node_1748279401.2858858_gc1 needs SQL generation for 'Grandchild 1...'

Selected node: node_1748279401.2858858_gc1

Expected: Should select a leaf node (grandchild) first, not the parent


## 8. Analyze Tree Structure Function

In [6]:
# Helper to visualize tree analysis
async def show_tree_analysis():
    """Show detailed tree analysis"""
    analysis = await task_checker.analyze_tree()
    
    print("Tree Analysis:")
    print(f"  Total nodes: {analysis.get('total_nodes', 0)}")
    print(f"  Processed (good): {len(analysis.get('processed_good', []))}")
    print(f"  Processed (poor): {len(analysis.get('processed_poor', []))}")
    print(f"  Unprocessed: {len(analysis.get('unprocessed', []))}")
    print(f"  Recommended action: {analysis.get('recommended_action', 'none')}")
    print(f"  Recommended node: {analysis.get('recommended_node', 'none')}")
    
    return analysis

# Show current tree analysis
analysis = await show_tree_analysis()

Tree Analysis:
  Total nodes: 4
  Processed (good): 0
  Processed (poor): 0
  Unprocessed: 4
  Recommended action: process
  Recommended node: node_1748279401.2858858_gc1


## 9. Test Error Handling

In [7]:
# Clear memory completely
await memory.clear()

# Don't initialize any tree
args = TaskStatusCheckerArgs(task="Check error handling for no tree")
result = await task_checker.run(args)
print("No tree error:")
print(result)
print("\nExpected: ACTION: ERROR indicating no query tree found")

No tree error:
STATUS REPORT:
- Error: No query tree found

ACTION: ERROR: No query tree found

Expected: ACTION: ERROR indicating no query tree found


## 10. Test Tool Interface with Pydantic Model

In [8]:
# Test the tool interface
print("Testing TaskStatusChecker as a tool...")
print("=" * 50)

# Get the tool
tool = task_checker.get_tool()

# Check tool properties
print(f"Tool name: {tool.name}")
print(f"Tool description: {tool.description}")
print(f"Args type: {tool._args_type}")
print(f"Return type: {tool._return_type}")

# Test creating arguments
args1 = TaskStatusCheckerArgs()  # Default empty task
args2 = TaskStatusCheckerArgs(task="Check specific task")

print(f"\nDefault args: {args1}")
print(f"Custom args: {args2}")

# Test the Pydantic model schema
schema = TaskStatusCheckerArgs.model_json_schema()
print(f"\nPydantic model schema:")
print(f"  Title: {schema.get('title')}")
print(f"  Properties: {list(schema.get('properties', {}).keys())}")
print(f"  Task field: {schema['properties']['task']}")

print("\n✅ Tool interface works correctly with Pydantic model!")

Testing TaskStatusChecker as a tool...
Tool name: task_status_checker
Tool description: Check task status and determine next action based on query tree state
Args type: <class 'task_status_checker.TaskStatusCheckerArgs'>
Return type: <class 'str'>

Default args: task=''
Custom args: task='Check specific task'

Pydantic model schema:
  Title: TaskStatusCheckerArgs
  Properties: ['task']
  Task field: {'default': '', 'description': 'The task instruction (optional)', 'title': 'Task', 'type': 'string'}

✅ Tool interface works correctly with Pydantic model!
